# High-resolution PM 2.5 and components data

The data is harnessed at high spatial resolution of 1km x 1km. After merging multiple sources and removing missing values, the dataset contains around 5,000,000 observations. 

The original data sources consist of rasters, sources are described below. Unfortunately, data from the NASA Earth Observatory must be downloaded manually after creating an account. 


### PM 2.5 Components Data, US 2000

- Source: NASA Earth Observatory
- URL: https://sedac.ciesin.columbia.edu/data/set/aqdh-pm2-5-component-ec-nh4-no3-oc-so4-50m-1km-contiguous-us-2000-2019/data-download
- Research Paper: https://www.researchsquare.com/article/rs-1745433/v2
- Files: 
  - `aqdh_pm25component_ec_2000_non_urban` (Elemental Carbon)
  - `aqdh_pm25component_nh4_2000_non_urban` (Ammonium)
  - `aqdh_pm25component_no3_2000_non_urban` (Nitrate)
  - `aqdh_pm25component_oc_2000_non_urban` (Organic Carbon)
  - `aqdh_pm25component_so4_2000_non_urban` (Sulfate)

### Total PM 2.5 Data, US 2000

- Source: NASA Earth Observatory
- URL: https://sedac.ciesin.columbia.edu/data/set/aqdh-pm2-5-annual-concentrations-1km-contiguous-us-2000-2019/data-download
- Research Paper: https://www.sciencedirect.com/science/article/pii/S0160412019300650?via%3Dihub#s0115
- Files;
  - `Annual-geotiff/2000.tif` (Total PM 2.5)

The spatial graph is obtained from the row-column representation in the raster's original projections.

*TODO*: More variables about land used and demographic will be added, but keep in mind that that the dataset is already very large. The majority of existing methods for spatial data scale polynomially with the number of observations, so the dataset is already beyond what spatial methods can handle.

For the code we assume that the above files are downloaded and uncompressed in a `data/` folder.

## Requirements

```yaml
# install with conda env create -f <file-name>.yaml and
# activate with conda activate pm25comps
name: pm25comps
channels:
  - defaults
  - conda-forge
dependencies:
  - python=3.10
  - pandas
  - pyarrow
  - lxml
  - pip
  - pip:
    - matplotlib
    - ipykernel
    - pyreadr
    - networkx
    - rasterio
    - tqdm
    - pyproj
    - lxml
```

In [13]:
import os
import pyreadr
import pyproj
import pandas as pd
import rasterio
import numpy as np
import networkx as nx
from tqdm import tqdm
import tempfile

## Load PM2.5 and components data

Utility function to map coordinates to row-column representation using a raster's original projection.

In [3]:
def update_dataframe_with_raster(df, raster_path):
    """
    Add row, col of raster into dataframe.

    Parameters:
    - df: pandas DataFrame with 'lon', 'lat', and 'value' columns
    - raster_path: path to the input raster file
    """

    # Read the CRS from the raster
    with rasterio.open(raster_path) as src:
        raster_crs = src.crs

    # If DataFrame coordinates are in WGS84 (can be replaced by other known CRS)
    data_crs = pyproj.CRS("EPSG:4326")

    # Initialize transformer between DataFrame CRS and raster CRS
    transformer = pyproj.Transformer.from_crs(data_crs, raster_crs, always_xy=True)

    # Transform DataFrame coordinates to raster CRS
    df["x"], df["y"] = transformer.transform(df["lon"].values, df["lat"].values)

    # Open the raster file to read its properties and data
    with rasterio.open(raster_path) as src:
        # Convert the x, y coordinates to row, col indices in the raster
        df["row"], df["col"] = zip(
            *[~src.transform * (x, y) for x, y in zip(df["x"], df["y"])]
        )

        # Round row and col to integers
        df["row"] = df["row"].astype(int)
        df["col"] = df["col"].astype(int)


Load and process components data

In [4]:
df_comps = {}
for comp in ["ec", "so4", "no3", "nh4", "oc"]:
    print(f"{comp}...", end="")
    df = pyreadr.read_r(f"data/aqdh_pm25component_{comp}_2000_non_urban.rds")[None]
    df.rename(columns={f"final.predicted.{comp}": "value"}, inplace=True)
    update_dataframe_with_raster(df, "data/Annual-geotiff/2000.tif")
    df = df[["row", "col", "value"]]
    df = df.groupby(["row", "col"]).mean().reset_index()
    df_comps[comp] = df.rename(columns={"value": f"value_{comp}"})
    print(" done.")
    print(df.head())


ec... done.
   row  col     value
0    0  897  0.192862
1    0  898  0.166689
2    0  899  0.185424
3    1  895  0.225625
4    1  896  0.191031
so4... done.
   row  col     value
0    0  897  0.932552
1    0  898  0.928306
2    0  899  0.989542
3    1  895  1.012710
4    1  896  0.980354
no3... done.
   row  col     value
0    0  897  0.327126
1    0  898  0.316236
2    0  899  0.334373
3    1  895  0.317554
4    1  896  0.272384
nh4... done.
   row  col     value
0    0  897  0.154040
1    0  898  0.143897
2    0  899  0.159752
3    1  895  0.140403
4    1  896  0.138589
oc... done.
   row  col     value
0    0  897  1.127481
1    0  898  1.123821
2    0  899  1.102654
3    1  895  1.115354
4    1  896  1.045228


Utility to convert raster to row-column data frame representation

In [5]:
def raster_to_dataframe(raster_path):
    """
    Convert raster to row column dataframe
    """
    with rasterio.open(raster_path) as src:
        array_data = src.read(1)
        na_entry = src.nodata
    height, width = array_data.shape
    row, col = np.indices((height, width))
    row = row.flatten()
    col = col.flatten()
    value = array_data.flatten()
    df = pd.DataFrame({"row": row, "col": col, "value": value})
    df = df[df["value"] != na_entry]
    return df

Read pm2.5 and merge with component data, remove incomplete observations

In [10]:
# load pm25 for 2000
df = raster_to_dataframe("data/Annual-geotiff/2000.tif").rename(columns={"value": "value_pm25"})

for c in df_comps:
    df = df.merge(df_comps[c], on=["row", "col"], how="outer")

df = df.dropna()
df.index = pd.Index(df["row"].astype(str) + "_" + df["col"].astype(str))
df.head()

,row,col,value_pm25,value_ec,value_so4,value_no3,value_nh4,value_oc
73_621,73,621,5.055394,0.206392,0.865740,0.264086,0.165173,1.023715
73_632,73,632,4.512170,0.293871,0.828311,0.300283,0.072935,1.526300
73_633,73,633,4.923641,0.207079,0.795750,0.288293,0.130885,1.111460
74_621,74,621,3.557837,0.147502,0.763437,0.213010,0.143760,0.939082
74_622,74,622,4.635057,0.126345,0.714855,0.196622,0.190206,0.938472


## Make graph

In [11]:
# make grid graph
max_rows = df["row"].max()
max_cols = df["col"].max()

edgelist = []

for r in tqdm(range(max_rows + 1)):
    for c in range(max_cols + 1):
        if c < max_cols:
            edgelist.append((f"{r}_{c}", f"{r}_{c+1}"))
        if r < max_rows:
            edgelist.append((f"{r}_{c}", f"{r+1}_{c}"))

G = nx.from_edgelist(edgelist)
G = nx.subgraph(G, df.index)

print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

100%|██████████| 2848/2848 [00:06<00:00, 458.75it/s]


Number of nodes: 4170738
Number of edges: 8304748


## Save data

In [17]:
edges = pd.DataFrame(edgelist, columns=["source", "target"])
coords = df[["row", "col"]]

tmpdir = tempfile.TemporaryDirectory().name
os.makedirs(f"{tmpdir}/graph", exist_ok=True)
os.makedirs(f"uploads/dataverse", exist_ok=True)

edges.to_parquet(f"{tmpdir}/graph/edges.parquet")
coords.to_parquet(f"{tmpdir}/graph/coords.parquet")

os.system(f"cd {tmpdir} && tar -czvf graph_pm25comps.tar.gz graph/edges.parquet graph/coords.parquet")
os.system(f"mv {tmpdir}/graph_pm25comps.tar.gz uploads/dataverse/graph_pm25comps.tar.gz")

# %%
df.drop(columns=["row", "col"]).to_parquet("uploads/dataverse/raster_pm25comps.parquet")

a graph/edges.parquet
a graph/coords.parquet
